In [ ]:
from causalnex.inference import InferenceEngine
from causalnex.network import BayesianNetwork
from sklearn.model_selection import train_test_split, KFold
import xml_read
import networkx as nx
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import log_loss
from var_name import dict_CE
import numpy as np
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif']=['SimHei']

In [ ]:
file_name = 'causal network'
graph_structure = xml_read.parse_xml_sm(file_name+'.xml')
data = pd.read_csv('Structural data_coarse-grained.csv')


In [ ]:
#calculate the initial g and Q for the X to AMI, i.e. the input file

def outcome_k_fold_fit_and_predict(data:pd.DataFrame,  num_folds:int):
    all_ami_probs = np.zeros(len(data))
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=99)
    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(data)):
        train, test = data.iloc[train_idx], data.iloc[test_idx]
        bn = BayesianNetwork(graph_structure)
        bn = bn.fit_node_states(data)
        bn_cpd = bn.fit_cpds(train, method="BayesianEstimator", bayes_prior="K2")
        ie = InferenceEngine(bn_cpd)
        calc_nodes0 = bn_cpd.nodes
        calc_nodes0.remove('AMI')
        calc_nodes1=['CVD', 'CFAB', 'Dys', 'ALB', 'ChestPain', 'Dyspnea', 'HGB', 'Numbness', 'STAB', 'TAB', 'Age','CRF', 'Sex','Nutrition','Lifestyle','HTN'];
        calc_nodes=calc_nodes0
        data_calc = test.loc[:, calc_nodes]
        gt = list(test.loc[:, 'AMI'])
        list_calc = []
        for idx in range(data_calc.shape[0]):
            row = data_calc.iloc[idx]
            list_calc.append(row.to_dict())    
        marginal_condition = ie.query(list_calc) 
        ami_preds = [one_pred['AMI'] for one_pred in marginal_condition]
        if isinstance(ami_preds[0], dict) and 1 in ami_preds[0]:
            ami_probs = np.array([pred[1] for pred in ami_preds])
            all_ami_probs[test_idx] = ami_probs
        else:
            print(f"Unexpected structure in ami_preds: {ami_preds[0]}")
            break
    return all_ami_probs   
Q=outcome_k_fold_fit_and_predict(data, 10)
print(Q)
df = pd.DataFrame({'Q_AMI': Q})
output_file = 'input_Q_AMI.xlsx'
df.to_excel(output_file, index=False) 

In [ ]:
def treatment_k_fold_fit_and_predict(data:pd.DataFrame,  num_folds:int,treatment:str):
    print(treatment)
    all_treatment_probs = np.zeros(len(data))
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=99)
    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(data)):
        train, test = data.iloc[train_idx], data.iloc[test_idx]
        bn = BayesianNetwork(graph_structure)
        bn = bn.fit_node_states(data)
        bn_cpd = bn.fit_cpds(train, method="BayesianEstimator", bayes_prior="K2")
        ie = InferenceEngine(bn_cpd)
        calc_nodes0 = bn_cpd.nodes
        calc_nodes0.remove('AMI')
        calc_nodes0.remove(treatment)
        calc_nodes=calc_nodes0#validation A
        data_calc = test.loc[:, calc_nodes]
        list_calc = []
        for idx in range(data_calc.shape[0]):
            row = data_calc.iloc[idx]
            list_calc.append(row.to_dict())       
        marginal_condition = ie.query(list_calc)           
        treatment_preds = [one_pred[treatment] for one_pred in marginal_condition]
        if isinstance(treatment_preds[0], dict) and 1 in treatment_preds[0]:
            treatment_probs = np.array([pred[1] for pred in treatment_preds])
            all_treatment_probs[test_idx] = treatment_probs
        else:
            print(f"Unexpected structure in treatment_preds: {treatment_preds[0]}")
            break
        print(all_treatment_probs)
    combinestr='input_g_'+treatment
    combinestr2='C:/yourdir/XtoAMI/g/'+combinestr+'.xlsx'
    df = pd.DataFrame({combinestr: all_treatment_probs})
    df.to_excel(combinestr2, index=False)
    return all_treatment_probs
calc_nodes=['Age','HTN','Sex','CRF','Dys','CFAB','CVD','Nutrition','Lifestyle'];
for i in range(len(calc_nodes)):    
    treatment_k_fold_fit_and_predict(data, 10,calc_nodes[i])
